# TsRelBreakTest

This is the module that performs relative, statisitcal tests to detect breaks between 2 parts (before and after a potential break time) of a candidate time series. We detect 2 types of breaks:
-Mean Breaks
-Variance breaks



In [1]:
from set_up import *
pd.options.display.max_rows=7
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# Specific for this module
from pybreaks.break_test import TsRelBreakTest

In [2]:
# this cell loads either test data with properties as set by the user or real 
# data and allows to introduce a break
def load_data(real=False, trend=0.01, bias=0.1, breaktime=datetime(2012,7,1), 
              size=(0.2, 1.1), resample=False):
    if real:
        CAN, REF = load_real_data(707393, breaktime, size, resample)
    else:
        CAN, REF = load_test_data(trend, bias, breaktime, size, resample, seed=1234)
    return CAN, REF

In [3]:
# this cell creates wrappers of the plotting functions
def plot_ts(test_obj):
    # add here the function from the base class for plotting
    fig,ax = plt.subplots(1,1)
    test_obj.plot_ts(frame=test_obj.df_original, title='Main DataFrame', ax=ax)
    ax.grid(color='w', linewidth=2, linestyle='solid')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    plt.close('all')
    return fig

def plot_stats_ts(test_obj):
    # add here the function from the base class for plotting
    test_obj.plot_ts(frame=test_obj.df_original, title='Main DataFrame')
    fig = test_obj.plot_stats_ts(test_obj.df_original, kind='line', 
                                 save_path=None, title='Main DataFrame Stats')
    plt.close('all')
    return fig

In [4]:
bias_corr_methods = ['linreg', 'mean_std', 'min_max', 'cdf_match']
@interact(TREND=(0, 0.05, 0.01), BIAS=(-0.5, 0.5, 0.1), BREAK_ADD=(-0.4, 0.4, 0.05), 
          BREAK_MULT=(0.7, 1.3, 0.05), BIAS_CORR_METHOD=bias_corr_methods)

def BreakTest(USE_REAL_DATA=False, TESTDATA_TREND=0.01, TESTDATA_BIAS=0.1, 
              BREAK_ADD=0., BREAK_MULT=1., 
              RESAMPLE=False, BIAS_CORR_METHOD='cdf_match'):
    
    BREAKTIME=datetime(2012,7,1)
    size=(BREAK_ADD, BREAK_MULT)

    CAN, REF = load_data(USE_REAL_DATA, TESTDATA_TREND, TESTDATA_BIAS, BREAKTIME, 
                         size, RESAMPLE)

    test = TsRelBreakTest(CAN, REF, BREAKTIME, test_check_spearR_sig=(0, 0.01),
                          test_resample=('M', 0.3) if not RESAMPLE else None, # always resample for testing!
                          bias_corr_method=BIAS_CORR_METHOD)
    
    isbreak, breaktype, testresult, errorcode = test.run_tests()
    _, error, checkstats = test.get_results()

    print('Type of detected break:{}'.format(breaktype if isbreak else 'No Break'))
    print(errorcode)
    print(error)
    print(checkstats)

    test.plot_ts(test.df_original) # todo: change colors and add a label of the break type to the TS plot
    test.plot_stats_ts(test.df_original, kind='line')
    

aW50ZXJhY3RpdmUoY2hpbGRyZW49KENoZWNrYm94KHZhbHVlPUZhbHNlLCBkZXNjcmlwdGlvbj11J1VTRV9SRUFMX0RBVEEnKSwgRmxvYXRTbGlkZXIodmFsdWU9MC4wMSwgZGVzY3JpcHRpb27igKY=
